<div style="display:fill;
           background-color:#DDDDDD;
           letter-spacing:0.5px;border-bottom: 2px solid black;">
<img src="https://raw.githubusercontent.com/IqmanS/Machine-Learning-Notebooks/main/optiver/optiver-banner.jpg">
    
<H1 style="padding: 20px; color:black; font-weight:600;font-family: 'Garamond', 'Lucida Sans', sans-serif; text-align: center; font-size: 36px;">📈 OPTIVER - TRADING AT THE CLOSE</H1>
</div>


In [ ]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
import os
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("dark") # Theme for plots as Dark
# sns.set_palette("viridis")
sns.color_palette("flare")
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import optuna
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from catboost import Pool, CatBoostRegressor, cv

import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

<div id="1" style="background-color: #DDDDDD; padding: 20px; border-radius: 20px; border: 2px solid black;">
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #000000; font-weight: bold; font-size: 42px;">
   Dataset Overview
    </h1>
</div>
<div style="padding: 20px;font-size: 16px; font-family: 'Verdana'; color: #222222; text-align: left;">This notebook expands upon the work done by @bzeniti: <a href="https://www.kaggle.com/code/bzeniti/simple-imputation-xgb-regressor" style="color: #222222;text-decoration: underline;">Notebook Link</a>.<br> Make sure to visit and upvote it. 
</div>

In [ ]:
train_data = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
test_data = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv")
test_target = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv")
sample_submission = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv")

train_data.head()

In [ ]:
train_data.shape

In [ ]:
train_data.info()

In [ ]:
train_data.isna().sum().sort_values(ascending = False)

<div id="2" style="background-color: #DDDDDD; padding: 20px; border-radius: 20px; border: 2px solid black;">
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #000000; font-weight: bold; font-size: 42px;">
   Simple Preprocessing
    </h1>
</div>

In [ ]:
def Preprocess1(data):
    data.set_index("row_id",inplace=True)
    data.drop(['far_price', 'near_price'], axis = 1,inplace=True)
    return data

train_data = Preprocess1(train_data)

In [ ]:
def Preprocess2(data):
    stock_list = list(data['stock_id'].unique())

    for stock in stock_list:
        stock_df = data.loc[data['stock_id'] == stock]
        imp_mean_imbalance_size = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_reference_price = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_matched_size = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_bid_price = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_ask_price = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_wap = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_target = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_seconds_in_bucket = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_imbalance_buy_sell_flag = SimpleImputer(missing_values=np.nan, strategy='mode')
        imp_mean_bid_size = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp_mean_ask_size = SimpleImputer(missing_values=np.nan, strategy='mean')

        data['imbalance_size'] = imp_mean_imbalance_size.fit_transform(np.array(data['imbalance_size']).reshape(-1,1))
        data['reference_price'] = imp_mean_reference_price.fit_transform(np.array(data['reference_price']).reshape(-1,1))
        data['matched_size'] = imp_mean_matched_size.fit_transform(np.array(data['matched_size']).reshape(-1,1))
        data['bid_price'] = imp_mean_bid_price.fit_transform(np.array(data['bid_price']).reshape(-1,1))
        data['ask_price'] = imp_mean_ask_price.fit_transform(np.array(data['ask_price']).reshape(-1,1))
        data['wap'] = imp_mean_wap.fit_transform(np.array(data['wap']).reshape(-1,1))
        data['target'] = imp_mean_target.fit_transform(np.array(data['target']).reshape(-1,1))
        data['seconds_in_bucket'] = imp_mean_seconds_in_bucket.fit_transform(np.array(data['seconds_in_bucket']).reshape(-1,1))
        data['bid_size'] = imp_mean_bid_size.fit_transform(np.array(data['bid_size']).reshape(-1,1))
        data['ask_size'] = imp_mean_ask_size.fit_transform(np.array(data['ask_size']).reshape(-1,1))

    return data, imp_mean_imbalance_size, imp_mean_reference_price, imp_mean_matched_size, imp_mean_bid_price, \
            imp_mean_ask_price, imp_mean_wap, imp_mean_seconds_in_bucket, imp_mean_bid_size, imp_mean_ask_size

def applyPreprocess2(data):
    data['imbalance_size'] = imp_mean_imbalance_size.transform(np.array(data['imbalance_size']).reshape(-1,1))
    data['reference_price'] = imp_mean_reference_price.transform(np.array(data['reference_price']).reshape(-1,1))
    data['matched_size'] = imp_mean_matched_size.transform(np.array(data['matched_size']).reshape(-1,1))
    data['bid_price'] = imp_mean_bid_price.transform(np.array(data['bid_price']).reshape(-1,1))
    data['ask_price'] = imp_mean_ask_price.transform(np.array(data['ask_price']).reshape(-1,1))
    data['wap'] = imp_mean_wap.transform(np.array(data['wap']).reshape(-1,1))
    data['seconds_in_bucket'] = imp_mean_seconds_in_bucket.transform(np.array(data['seconds_in_bucket']).reshape(-1,1))
    data['bid_size'] = imp_mean_bid_size.transform(np.array(data['bid_size']).reshape(-1,1))
    data['ask_size'] = imp_mean_ask_size.transform(np.array(data['ask_size']).reshape(-1,1))
    
    return data

train_data, imp_mean_imbalance_size, imp_mean_reference_price, imp_mean_matched_size, imp_mean_bid_price, imp_mean_ask_price, imp_mean_wap, imp_mean_seconds_in_bucket, imp_mean_bid_size, imp_mean_ask_size = Preprocess2(train_data)

In [ ]:
train_data.isna().sum()

<div id="4" style="background-color: #DDDDDD; padding: 20px; border-radius: 20px; border: 2px solid black;">
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #000000; font-weight: bold; font-size: 42px;">
   Training Models
    </h1>
</div>

In [ ]:
y = train_data['target']
X = train_data.drop(['target'], axis = 1)
X = X.drop(['time_id'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 21)

<div id="4.1" >
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #263A29; font-weight: bold; font-size: 36px;">
   4.1 Baseline XGB Model
    </h1>
</div>
<hr>

In [ ]:
xgb = XGBRegressor(n_estimators = 100,tree_method="gpu_hist")
print("CV score of XGB is ",-1*cross_val_score(xgb,X,y,cv=2, scoring = 'neg_mean_absolute_error').mean())

<div id="4.2" >
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #263A29; font-weight: bold; font-size: 36px;">
   4.2 Baseline CatBoost Model
    </h1>
</div>
<hr>

In [ ]:
cat = CatBoostRegressor(n_estimators = 100,verbose=False)
print("CV score of CAT is ",-1*cross_val_score(cat,X,y,cv=2, scoring = 'neg_mean_absolute_error').mean())

<div id="4.3" >
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #263A29; font-weight: bold; font-size: 36px;">
   4.3 Baseline LGBM Model
    </h1>
</div>
<hr>

In [ ]:
lgbm = LGBMRegressor(n_estimators = 100, verbose=0, force_col_wise=True, device="gpu")
print("CV score of LGBM is ",-1*cross_val_score(lgbm,X,y,cv=2, scoring = 'neg_mean_absolute_error').mean())

<div id="4.4" >
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #263A29; font-weight: bold; font-size: 36px;">
   4.4 MAE of XGB+LGBM+CatBoost
    </h1>
</div>
<hr>

In [ ]:
xgb.fit(X_train,y_train)
cat.fit(X_train,y_train)
lgbm.fit(X_train,y_train)

pred = 0.3*xgb.predict(X_test) + 0.3*cat.predict(X_test) + 0.40*lgbm.predict(X_test)
mae = mean_absolute_error(y_test, pred)
print(f"MAE of merged predictions: {mae}")

<div id="4.4" >
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #263A29; font-weight: bold; font-size: 36px;">
   4.4 Preds on Example 'test.csv'
    </h1>
</div>
<hr>

In [ ]:
test_data = Preprocess1(test_data)
test_data.drop(['time_id'], axis = 1,inplace=True)
test_data.head()

In [ ]:
test_target.head()

In [ ]:
y_sample_pred =  0.3*xgb.predict(test_data) + 0.3*cat.predict(test_data) + 0.40*lgbm.predict(test_data)
y_sample_true = test_target['revealed_target'].dropna()

In [ ]:
mae_test = mean_absolute_error(y_sample_pred, y_sample_true)
print(f"MAE on the test Dataset: {mae_test}")

<div id="5" style="background-color: #DDDDDD; padding: 20px; border-radius: 20px; border: 2px solid black;">
    <h1 style="font-family:  'Garamond', 'Lucida Sans', sans-serif; text-align: center; color: #000000; font-weight: bold; font-size: 42px;">
   Submission from API
    </h1>
</div>

In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print("Submitting Predictions")

    test_df = Preprocess1(test)
    test_df = applyPreprocess2(test_df)
    sample_prediction['target'] =  0.3*xgb.predict(test_df) + 0.3*cat.predict(test_df) + 0.40*lgbm.predict(test_df)
    env.predict(sample_prediction)
    counter += 1

In [ ]:
print(sample_prediction.head(20))